# 数据清洗、转换、合并、重塑

In [1]:
import pandas as pd
import numpy as np

## 1、合并数据集

## 1.1、merge用法
**merge可根据一个或多个键将不同的dataframe中的行连接起来**

In [3]:
df1 = pd.DataFrame({'data1':range(3),'key1':['a','b','d']})
df2 = pd.DataFrame({'data2':range(7),'key1':['b','b','a','c','a','a','b']})
print(df1)
print(df2)

   data1 key
0      0   a
1      1   b
2      2   d
   data2 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   a
6      6   b


In [10]:
# 没有指定，因此默认重复的列，作为链接键
print(pd.merge(df1,df2))

# 显式地指定某个共同键，作为链接
print(pd.merge(df1,df2,on='key'))

   data1 key  data2
0      0   a      2
1      0   a      4
2      0   a      5
3      1   b      0
4      1   b      1
5      1   b      6
   data1 key  data2
0      0   a      2
1      0   a      4
2      0   a      5
3      1   b      0
4      1   b      1
5      1   b      6


In [30]:
# 如果键名不一样，可分别指定
print(pd.merge(df1,df2,left_on='data1',right_on='data2'))

# print(pd.merge(df1,df2,on='key'))的等价写法
print(pd.merge(df1,df2,left_on='key',right_on='key'))

   data1 key_x  data2 key_y
0      0     a      0     b
1      1     b      1     b
2      2     d      2     a
   data1 key  data2
0      0   a      2
1      0   a      4
2      0   a      5
3      1   b      0
4      1   b      1
5      1   b      6


In [35]:
# how的值 默认为 inner 即两者的交集

#  how: left 即以df1的dataframe为主
print(pd.merge(df1,df2,on='key',how='left'))

#  how: right 即以df2的dataframe为主
print(pd.merge(df1,df2,on='key',how='right'))

#  how: outer 即取并集
print(pd.merge(df1,df2,on='key',how='outer'))

   data1 key  data2
0      0   a    2.0
1      0   a    4.0
2      0   a    5.0
3      1   b    0.0
4      1   b    1.0
5      1   b    6.0
6      2   d    NaN
   data1 key  data2
0    0.0   a      2
1    0.0   a      4
2    0.0   a      5
3    1.0   b      0
4    1.0   b      1
5    1.0   b      6
6    NaN   c      3
   data1 key  data2
0    0.0   a    2.0
1    0.0   a    4.0
2    0.0   a    5.0
3    1.0   b    0.0
4    1.0   b    1.0
5    1.0   b    6.0
6    2.0   d    NaN
7    NaN   c    3.0


In [46]:
df3 = pd.DataFrame({'key1':['kv1','kv2','kv3'],'key2':['kvv1','kvv2','kvv3'],'lval':[1,2,3]})
df4 = pd.DataFrame({'key1':['kv1','kv2','kv3','kv1'],'key2':['kvv1','kvv2','kvv3','kvv4'],'rval':[1,3,2,4]})
print(df3)
print(df4)

  key1  key2  lval
0  kv1  kvv1     1
1  kv2  kvv2     2
2  kv3  kvv3     3
  key1  key2  rval
0  kv1  kvv1     1
1  kv2  kvv2     3
2  kv3  kvv3     2
3  kv1  kvv4     4


In [50]:
# 以多个相同的键明
print( pd.merge(df3,df4,on=['key1','key2']) )

  key1  key2  lval  rval
0  kv1  kvv1     1     1
1  kv2  kvv2     2     3
2  kv3  kvv3     3     2


In [61]:
# 索引上的合并
df5 = pd.DataFrame({'key':['a','b','a','a','b','c'],'value':range(6)})
df6 = pd.DataFrame({'data':[3,4]},index=['a','b'])

# 即左边以key为依据，右边以df6的索引值为依据
print(pd.merge(df5,df6,left_on='key',right_index=True))

# 之间以df5/df6的索引值为依据
df5.join(df6,how='outer')

  key  value  data
0   a      0     3
2   a      2     3
3   a      3     3
1   b      1     4
4   b      4     4


,key,value,data
0,a,0.0,NaN
1,b,1.0,NaN
2,a,2.0,NaN
3,a,3.0,NaN
4,b,4.0,NaN
5,c,5.0,NaN
a,NaN,NaN,3.0
b,NaN,NaN,4.0


## 1.2、concat
**concat可以沿着一条轴将多个对象堆叠到一起**

In [89]:
# 各个series不同的情况
s1 = pd.Series([1,2],index=['a','b'])
s2 = pd.Series([3,4,5],index=['c','d','e'])
s3 = pd.Series([6,7],index=['f','g'])

# axis=0 连接后类型不变
# print(pd.concat([s1,s2,s3]))

# axis=1 连接后的类型为 dataframe
# print(pd.concat([s1,s2,s3],axis=1,sort=True))


# 相同索引的series
s1 = pd.Series([1,2],index=['a','b'])
s2 = pd.Series([3,4],index=['a','b'])
# 合并时指定 dataframe列索引
tmp = pd.concat([s1,s2],axis=1,keys=['bb','cc'])
print(type(tmp))
print(tmp)

<class 'pandas.core.frame.DataFrame'>
   bb  cc
a   1   3
b   2   4


In [94]:
# 操作dataframe
p1 = pd.DataFrame(np.arange(6).reshape(3,2),index=['a','b','c'],columns=['one','two'])
p2 = pd.DataFrame(np.arange(4).reshape(2,2),index=['a','c'],columns=['three','four'])
pd.concat([p1,p2],sort=True)

,four,one,three,two
a,NaN,0.0,NaN,1.0
b,NaN,2.0,NaN,3.0
c,NaN,4.0,NaN,5.0
a,1.0,NaN,0.0,NaN
c,3.0,NaN,2.0,NaN


## 2、旋转
**stack: 将数据的列'旋转'为行**

**unstack: 将数据的行'旋转'为列**

*即实现series与dataframe相互转换*

In [102]:
p1 = pd.DataFrame(np.arange(4).reshape(2,2),columns=['a','b'])
print(p1)

# 转换成series
tmp = p1.stack()
print(type(tmp))
print(tmp)

# 转换成dataframe
tmp = tmp.unstack()
print(type(tmp))
print(tmp)


   a  b
0  0  1
1  2  3
<class 'pandas.core.series.Series'>
0  a    0
   b    1
1  a    2
   b    3
dtype: int32
<class 'pandas.core.frame.DataFrame'>
   a  b
0  0  1
1  2  3


## 3、数据转换

In [108]:
# 移除重复数据
data = pd.DataFrame({'k1':['a']*3+['b']*4,'k2':[1,1,2,2,3,4,4]})
print(data)

# 判断是否出现重复值
# duplicated 返回一个布尔类型的series，以第一个值为依据
print(data.duplicated())

# 删除重复的值，保留相同的第一个
print(data.drop_duplicates())

# 删除重复的值，保留相同的最后一个
print(data.drop_duplicates(keep='last'))

# 删除指定列，保留相同的最后一个
print(data.drop_duplicates(['k1'],keep='last'))

  k1  k2
0  a   1
1  a   1
2  a   2
3  b   2
4  b   3
5  b   4
6  b   4
0    False
1     True
2    False
3    False
4    False
5    False
6     True
dtype: bool
  k1  k2
0  a   1
2  a   2
3  b   2
4  b   3
5  b   4
  k1  k2
1  a   1
2  a   2
3  b   2
4  b   3
6  b   4
  k1  k2
2  a   2
6  b   4


In [111]:
# 使用函数进行转换
data = pd.DataFrame({'name':['a','b','c','d'],'sex':['boy','girl','boy','boy']})
# 增加一列，根据name的值，获取每个人的分数
scores = {
    'a' : 90,
    'b' : 80,
    'c' : 75,
    'd' : 70
}
data['score'] = data['name'].map(scores)
print(data)

# 传入匿名函数
data['score'] = data['name'].map(lambda x: scores[x])
print(data)


  name   sex  score
0    a   boy     90
1    b  girl     80
2    c   boy     75
3    d   boy     70
  name   sex  score
0    a   boy     90
1    b  girl     80
2    c   boy     75
3    d   boy     70


In [115]:
# 值替换

data = pd.Series([1,100,2,100,3])

# 把100 替换成4
print(data.replace(100,4))


# 区间个数统计
ages = [20,22,21,25,30,21,35,45,50,60]
bi = [18,25,30,60]
cuts = pd.cut(ages,bi)
print(cuts)
print(pd.value_counts(cuts))

0    1
1    4
2    2
3    4
4    3
dtype: int64
[(18, 25], (18, 25], (18, 25], (18, 25], (25, 30], (18, 25], (30, 60], (30, 60], (30, 60], (30, 60]]
Categories (3, interval[int64]): [(18, 25] < (25, 30] < (30, 60]]
(18, 25]    5
(30, 60]    4
(25, 30]    1
dtype: int64


## 4、字符串函数

In [127]:
s1 = pd.Series({'a':'hello world','b':'world','c':'hello','d':'hello haha'})
print(s1)

# 是否包含
print(s1.str.contains('hello'))

s2 = pd.Series([1,1,2,1,2,3,4],dtype='str')
# 出现的次数
print(s2.str.count('1'))

a    hello world
b          world
c          hello
d     hello haha
dtype: object
a     True
b    False
c     True
d     True
dtype: bool
0    1
1    1
2    0
3    1
4    0
5    0
6    0
dtype: int64
